<a href="https://colab.research.google.com/github/hchgssarwyh/ai_fiction/blob/features/tg%2Bmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telegram bot

In [1]:
pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=d420c13a5a18bd8b25e3b564a93094157f7b1060cc7f42f2a4734c8ed29040a5
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [2]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pickle

import telebot;
from telebot import types



In [3]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [4]:
def evaluate1(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    with open('idx_to_char_hokku.pickle', 'rb') as f:
      idx_to_char = pickle.load(f)
    with open('char_to_idx_hokku.pickle', 'rb') as f:
      char_to_idx = pickle.load(f)
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [5]:
def evaluate2(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    with open('idx_to_char_harms.pickle', 'rb') as f:
      idx_to_char = pickle.load(f)
    with open('char_to_idx_harms.pickle', 'rb') as f:
      char_to_idx = pickle.load(f)
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [6]:
def evaluate3(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    with open('idx_to_char_sesenin.pickle', 'rb') as f:
      idx_to_char = pickle.load(f)
    with open('char_to_idx_sesenin.pickle', 'rb') as f:
      char_to_idx = pickle.load(f)
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
with open('idx_to_char_hokku.pickle', 'rb') as f:
  idx_to_char_hokku = pickle.load(f)
model_hokku = TextRNN(input_size=len(idx_to_char_hokku), hidden_size=128, embedding_size=128, n_layers=2)
model_hokku.load_state_dict(torch.load('model_hokku',map_location=torch.device('cpu')))
model_hokku.eval()

TextRNN(
  (encoder): Embedding(70, 128)
  (lstm): LSTM(128, 128, num_layers=2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=128, out_features=70, bias=True)
)

In [14]:
with open('idx_to_char_harms.pickle', 'rb') as f:
  idx_to_char_harms = pickle.load(f)
model_harms = TextRNN(input_size=len(idx_to_char_harms), hidden_size=128, embedding_size=128, n_layers=2)
model_harms.load_state_dict(torch.load('model_harms',map_location=torch.device('cpu')))
model_harms.eval()

TextRNN(
  (encoder): Embedding(84, 128)
  (lstm): LSTM(128, 128, num_layers=2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=128, out_features=84, bias=True)
)

In [ ]:
with open('idx_to_char_sesenin.pickle', 'rb') as f:
  idx_to_char_sesenin = pickle.load(f)
model_sesenin = TextRNN(input_size=len(idx_to_char_sesenin), hidden_size=128, embedding_size=128, n_layers=2)
model_sesenin.load_state_dict(torch.load('model_sesenin',map_location=torch.device('cpu')))
model_sesenin.eval()

In [16]:

model_hokku.to(device)
model_harms.to(device)
model_sesenin.to(device)

TextRNN(
  (encoder): Embedding(105, 128)
  (lstm): LSTM(128, 128, num_layers=2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=128, out_features=105, bias=True)
)

In [31]:
def poem_hokku(start):
  with open('idx_to_char_hokku.pickle', 'rb') as f:
    idx_to_char_hokku = pickle.load(f)
  with open('char_to_idx_hokku.pickle', 'rb') as f:
    char_to_idx_hokku = pickle.load(f)
  s = evaluate1(
      model_hokku, 
      char_to_idx_hokku, 
      idx_to_char_hokku, 
      temp=0.3, 
      prediction_len=200, 
      start_text=str(start)+' '
    )
  return s

In [18]:
def poem_harms(start):
  with open('idx_to_char_harms.pickle', 'rb') as f:
    idx_to_char_harms = pickle.load(f)
  with open('char_to_idx_harms.pickle', 'rb') as f:
    char_to_idx_harms = pickle.load(f)
  print(evaluate2(
      model_harms, 
      char_to_idx_harms, 
      idx_to_char_harms, 
      temp=0.3, 
      prediction_len=200, 
      start_text=str(start)+' '
    )
)

In [19]:
def poem_sesenin(start):
  with open('idx_to_char_sesenin.pickle', 'rb') as f:
    idx_to_char_sesenin = pickle.load(f)
  with open('char_to_idx_sesenin.pickle', 'rb') as f:
    char_to_idx_sesenin = pickle.load(f)
  print(evaluate3(
      model_sesenin, 
      char_to_idx_sesenin, 
      idx_to_char_sesenin, 
      temp=0.3, 
      prediction_len=200, 
      start_text=str(start)+' '
    )
)

In [32]:
bot = telebot.TeleBot('5912676709:AAFogJDypnO9nA9GZ9bKMB4Kw4ul_d7CYQk');
word = '';

@bot.message_handler(content_types=['text'])
def get_first_messages(message):


    if message.text == "/help":
        bot.send_message(message.from_user.id, 'Список команд: \n /author - выбор стиля для стиха или прозы \n /poem - выбор первого слова для начала стиха')
    elif message.text == "/poem":
        bot.send_message(message.from_user.id, "Введи первое слово")
        bot.register_next_step_handler(message, get_word);
    
    elif message.text == "/author":

        keyboard_author = types.InlineKeyboardMarkup(); #наша клавиатура

        key_hokku = types.InlineKeyboardButton(text='Хокку', callback_data='hokku');
        keyboard_author.add(key_hokku); #добавляем кнопку c хокку
        key_esenin= types.InlineKeyboardButton(text='Есенин', callback_data='esenin');
        keyboard_author.add(key_esenin);#добавляем кнопку c Есенином
        key_harms= types.InlineKeyboardButton(text='Хармс', callback_data='harms');
        keyboard_author.add(key_harms);#добавляем кнопку c Хармсом

        header = 'Список стилей';
        bot.send_message(message.from_user.id, text=header, reply_markup=keyboard_author)

    elif message.text == "/start":
        bot.send_message(message.from_user.id,'Отправь /poem и я сочиню для тебя что то!' )
    else:
        bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")


def get_word(message): #получаем слово
    global word;
    word = message.text;

    bot.send_message(message.from_user.id, word + " я люблю \n И кушать кашу я люблю \n Вот такой вот белый стих")

    keyboard_word = types.InlineKeyboardMarkup(); #наша клавиатура

    key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes'); #кнопка «Да»
    keyboard_word.add(key_yes); #добавляем кнопку в клавиатуру
    key_no= types.InlineKeyboardButton(text='Нет', callback_data='no');
    keyboard_word.add(key_no);
    question = 'Попробовать еще?';
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard_word)

    
def get_word_hokku(message): #получаем слово
    global word;
    word = message.text;

    bot.send_message(message.from_user.id, poem_hokku(word + " "))

    keyboard_word = types.InlineKeyboardMarkup(); #наша клавиатура

    key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes'); #кнопка «Да»
    keyboard_word.add(key_yes); #добавляем кнопку в клавиатуру
    key_no= types.InlineKeyboardButton(text='Нет', callback_data='no');
    keyboard_word.add(key_no);
    question = 'Попробовать еще?';
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard_word)

def get_word_harms(message): #получаем слово
    global word;
    word = message.text;

    bot.send_message(message.from_user.id, poem_harms(word + " "))

    keyboard_word = types.InlineKeyboardMarkup(); #наша клавиатура

    key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes'); #кнопка «Да»
    keyboard_word.add(key_yes); #добавляем кнопку в клавиатуру
    key_no= types.InlineKeyboardButton(text='Нет', callback_data='no');
    keyboard_word.add(key_no);
    question = 'Попробовать еще?';
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard_word)

def get_word_sesenin(message): #получаем слово
    global word;
    word = message.text;

    bot.send_message(message.from_user.id, poem_sesenin(word ))

    keyboard_word = types.InlineKeyboardMarkup(); #наша клавиатура

    key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes'); #кнопка «Да»
    keyboard_word.add(key_yes); #добавляем кнопку в клавиатуру
    key_no= types.InlineKeyboardButton(text='Нет', callback_data='no');
    keyboard_word.add(key_no);
    question = 'Попробовать еще?';
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard_word)

@bot.callback_query_handler(func=lambda call: call.data == "yes" or call.data == 'no')

def callback_worker(call):
    if call.data == "yes": #call.data это callback_data, которую мы указали при объявлении кнопки
        bot.send_message(call.message.chat.id, 'Введи первое слово');
        bot.register_next_step_handler(call.message, get_word);
    elif call.data == "no":
        bot.send_message(call.message.chat.id, ':(');

@bot.callback_query_handler(func=lambda call: call.data == "hokku" or call.data == 'esenin' or call.data == 'harms')
def callback_author(call):
    if call.data == "hokku": 
        bot.send_message(call.message.chat.id, 'Введи первое слово');

        bot.register_next_step_handler(call.message, get_word_hokku);
    elif call.data == "esenin":
        bot.send_message(call.message.chat.id, 'Введи первое слово');
        bot.register_next_step_handler(call.message, get_word_sesenin);
    elif call.data == "harms":
        bot.send_message(call.message.chat.id, 'Введи первое слово');
        bot.register_next_step_handler(call.message, get_word_harms);



In [ ]:
bot.polling(none_stop=True, interval=0) 

In [20]:
#poem_harms('всем привет')

всем привет   и нас
 Прозвонить бульдог,
 а по дороге,
 по панели,
 бегал Мишка
 по панели
 и кричал он:
 «Жу-жу-жу!
 Я теперь уже не Мишка,
 я совой пор делившись показ
     потом начинается голод,
 Воздух с крю
